In [1]:
def extraer_informacion(url, archivo_csv):
    #Importamos librerias
    import requests
    from bs4 import BeautifulSoup
    import urllib.parse
    import csv

    # Crea una lista para almacenar los enlaces
    link_list = []

    # Establece un límite para el número de páginas a procesar
    limit = 500

    # Establece el offset inicial
    of = 0

    while of<limit:
        # Realiza una solicitud HTTP a la página web con el offset actual
        response = requests.get(f"{url}/recent-submissions?offset={of}")

        # Obtiene el HTML de la respuesta
        html = response.text

        # Crea un objeto BeautifulSoup a partir del HTML
        soup = BeautifulSoup(html, "html.parser")
        
        # Parsea la URL
        parsed_url = urllib.parse.urlparse("{}/recent-submissions?offset={}".format(url, of))
        # Obtiene el contenido principal de la URL (todo antes del /repositorio)
        url_base = parsed_url.scheme + '://' + parsed_url.netloc

        # Busca todos los enlaces <a> en el HTML con clase image-link 
        # Esto permite buscar los hrefs que estan ligados a las imagenes(thumbnails) de cada tesis
        # Solo funciona si el repositorio usa thumbnails
        links = soup.find_all("a",{"class":"image-link"})
        hrefs1 = [url_base + a["href"] for a in links]

        # Itera sobre cada enlace y agrega su atributo "href" a la lista
        for link in hrefs1:
            link_list.append(link)

        # Incrementa el offset para la siguiente iteración
        of += 20

    # Acesso a la tabla de metadatos añadiendo ?show=full
    links_full = [link + '?show=full' for link in link_list]

    # Abre un archivo CSV en modo escritura
    with open(archivo_csv, 'w', newline='') as csvfile:
        # Crea el objeto escritor
        writer = csv.writer(csvfile)
        # Escribe la cabecera del archivo CSV
        writer.writerow(['Autores', 'Asesores', 'Título', 'Resumen', 'Año', 'Institución', 'Grado'])

        # Recorre cada link
        for link in links_full:
            # Descarga el contenido del link
            html = requests.get(link).text
            # Parsea el contenido del link
            soup = BeautifulSoup(html, 'html.parser')

            # Obtiene la información de cada link
            
            # Autores: como pueden ser varios se usa un bucle for
            authors = [element['content'] for element in soup.find_all('meta', {'name': 'citation_author'})]
            
            # Asesores y Grado: se usa find_next_sibling porque existe un problema de identificación con la etiqueta "td"
            asesores = [element.find_next_sibling().text for element in soup.find_all('td', string='dc.contributor.advisor')]
            grado = [element.find_next_sibling().text for element in soup.find_all('td', string='thesis.degree.name')]
            
            # Titulo, Resumen, Año e Institución: cuando la etiqueta es "meta" es más fácil de encontrar
            titulo = soup.find('meta', {'name': 'citation_title'})['content']
            abstract = soup.find('meta', {'name': "DCTERMS.abstract"})['content']
            año = soup.find('meta', {'name': "citation_date"})['content']
            institucion = soup.find('meta', {'name': 'citation_publisher'})['content']
            

            # Escribe la fila en el archivo CSV
            writer.writerow([authors, asesores, titulo, abstract, año, institucion, grado])

In [46]:
#extraemos 3 csv de los 3 repositorios
extraer_informacion("https://repositorio.esan.edu.pe/handle/20.500.12640/1814", "esan_repo.csv")

In [ ]:
extraer_informacion("https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757", "pucp_repo.csv")

In [47]:
extraer_informacion("https://repositorioacademico.upc.edu.pe/handle/10757/621395", "upc_repo.csv")